# Библиотеки Python для DS (семинары)

Урок 4. Практическая работа. Построение модели бинарной классификации

Используя diabetes-dataset, выполните следующие шаги:

1. Предварительная обработка данных:
— Очистите данные от пропусков (если они есть), обработайте аномалии и нормализуйте данные, если это необходимо.
— Разделите датасет на обучающую и тестовую выборки.

2. Построение модели:
— Используя алгоритм логистической регрессии, постройте модель бинарной классификации для предсказания наличия диабета у пациентов.
— Обучите модель на обучающем наборе данных.

3. Оценка модели:
— Вычислите и проанализируйте метрики точности (accuracy), точности (precision), полноты (recall) и F1-меру для обученной модели на тестовых данных.

## Выполнение домашней работы

In [2]:
# Библиотеки для работы с датасетом и графиками
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

##### Используя diabetes-dataset

In [5]:
# Считываем предоставленый для домашней работы датасет
df_diaD = pd.read_csv('diabetes/diabetes-dataset.csv')

##### 1. Предварительная обработка данных:
— Очистите данные от пропусков (если они есть), обработайте аномалии и нормализуйте данные, если это необходимо.

— Разделите датасет на обучающую и тестовую выборки.

In [6]:
# Выводим дата фрейм
df_diaD.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,2,138,62,35,0,33.6,0.127,47,1
1,0,84,82,31,125,38.2,0.233,23,0
2,0,145,0,0,0,44.2,0.630,31,1
3,0,135,68,42,250,42.3,0.365,24,1
4,1,139,62,41,480,40.7,0.536,21,0


Этот фрейм данных содержит следующие столбцы:

1. MedInc (Средний доход населения в квартале)
2. HouseAge (Средний возраст дома в квартале)
3. AveRooms (Средние количество команат)
4. AveBedrms (Средние количество спален)
5. Population (Население квартала)
6. AveOccup (Средняя заполняемость дома)
7. Latitude (Широта жилого квартала)
8. Longitude (Долгота жилого квартала)
9. MedHouseVal (Средняя цена дома)

In [6]:
# Выводим общую информацию
df_hous.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [7]:
# Смотрим тип данных
df_hous.dtypes

MedInc         float64
HouseAge       float64
AveRooms       float64
AveBedrms      float64
Population     float64
AveOccup       float64
Latitude       float64
Longitude      float64
MedHouseVal    float64
dtype: object

Из общей информации следует:
1. Всего 8 колонок
2. Нулевые значения отсуствуют
3. Нет пропущеных записей в колонках
4. Все колонки находятся в дробных float64 значениях

Статистический анализ объектных переменных не проводим так как их нет в данном датасете и нам не нужно их назначать согласно задания

In [8]:
# Статистический анализ числовых переменных
df_hous.describe().T

,count,mean,std,min,25%,50%,75%,max
MedInc,20640.0,3.870671,1.899822,0.499900,2.563400,3.534800,4.743250,15.000100
HouseAge,20640.0,28.639486,12.585558,1.000000,18.000000,29.000000,37.000000,52.000000
AveRooms,20640.0,5.429000,2.474173,0.846154,4.440716,5.229129,6.052381,141.909091
AveBedrms,20640.0,1.096675,0.473911,0.333333,1.006079,1.048780,1.099526,34.066667
Population,20640.0,1425.476744,1132.462122,3.000000,787.000000,1166.000000,1725.000000,35682.000000
AveOccup,20640.0,3.070655,10.386050,0.692308,2.429741,2.818116,3.282261,1243.333333
Latitude,20640.0,35.631861,2.135952,32.540000,33.930000,34.260000,37.710000,41.950000
Longitude,20640.0,-119.569704,2.003532,-124.350000,-121.800000,-118.490000,-118.010000,-114.310000
MedHouseVal,20640.0,2.068558,1.153956,0.149990,1.196000,1.797000,2.647250,5.000010


На основе представленных статистических данных неприрывных признаков можно сделать следующие выводы:

1. MedInc (Средний доход населения в квартале): 
- Средний общий рейтинг на уровне 3.53
- Миниманая 0.49
- Максимальная 15.00
- Медиана равна 3.87
- Отклонение состовляет 1.89
2. HouseAge (Средний возраст дома в квартале):
- Средние Счёт/Оценка, счастья на уровне 29.00
- Миниманая 1.00
- Максимальная 52.00
- Медиана равна 28.63
- Отклонение состовляет 12.58
3. AveRooms (Средние количество команат):
- Средние значение ВВП на душу населения на уровне 5.22
- Миниманая 0.84
- Максимальная 141.90
- Медиана равна 5.42
- Отклонение состовляет 2.47
4. AveBedrms (Средние количество спален):
- Средние социальной поддержки на уровне 1.04
- Миниманая 0.33
- Максимальная 34.06
- Медиана равна 1.09
- Отклонение состовляет 0.47
5. Population (Население квартала):
- Средние значение продолжительность здоровой жизни на уровне 1166.00
- Миниманая 3.00
- Максимальная 35682.00
- Медиана равна 1425.47
- Отклонение состовляет 1132.46
6. AveOccup (Средняя заполняемость дома):
- Средние значение Свободы выбора в жизни на уровне 2.81
- Миниманая 0.69
- Максимальная 1243.33
- Медиана равна 3.07
- Отклонение состовляет 10.38
7. Latitude (Широта жилого квартала):
- Средние значение щедрости на уровне 34.26
- Миниманая 32.54
- Максимальная 41.95
- Медиана равна 35.63
- Отклонение состовляет 2.13
8. Longitude (Долгота жилого квартала):
- Средние значение Восприятие корупции на уровне -118.49
- Миниманая -124.35
- Максимальная -114.31
- Медиана равна -119.56
- Отклонение состовляет 2.00
9. MedHouseVal (Средняя цена дома)
- Средние значение Восприятие корупции на уровне 1.79
- Миниманая 0.14
- Максимальная 5.00
- Медиана равна 2.06
- Отклонение состовляет 1.15

##### 2. Построение модели:

— Используя алгоритм логистической регрессии, постройте модель бинарной классификации для предсказания наличия диабета у пациентов.

— Обучите модель на обучающем наборе данных.

In [ ]:
# Установка стиля графиков
sns.set(style="whitegrid")

# Визуализация распределения нескольких выбранных числовых переменных
selected_columns = ['MedInc', 'HouseAge', 'AveRooms', 'AveOccup', 'MedHouseVal']

plt.figure(figsize=(15, 10))

for i, column in enumerate(selected_columns):
    plt.subplot(2, 3, i + 1)
    sns.histplot(df_hous[column], kde=True, bins=30)
    plt.title(f'Распределение {column}')
    plt.xlabel(column)
    plt.ylabel('Частота')

plt.tight_layout()
plt.show()

In [ ]:
# Визуализация корреляции между выбранными переменными
correlation_matrix = df_hous[selected_columns].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
plt.title('Корреляционная матрица')
plt.show()

##### 4. Разделите данные на обучающий и тестовый набор в соотношении 80/20.

In [11]:
# Разделение данных на обучающий и тестовый набор
X = df_hous[['MedInc', 'HouseAge']]
y = df_hous['MedHouseVal']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Размер обучающего набора:", X_train.shape)
print("Размер тестового набора:", X_test.shape)

Размер обучающего набора: (16512, 2)
Размер тестового набора: (4128, 2)


In [13]:
from sklearn.linear_model import LinearRegression

# Создание модели линейной регрессии
model = LinearRegression()

# Обучение модели на обучающем наборе
model.fit(X_train, y_train)

# Предсказания на тестовом наборе
y_pred = model.predict(X_test)

print("Предсказанные значения:", y_pred)

Предсказанные значения: [1.05551637 1.5113978  2.30731557 ... 4.32725202 1.72627159 1.72652128]


##### 3. Оценка модели:

— Вычислите и проанализируйте метрики точности (accuracy), точности (precision), полноты (recall) и F1-меру для обученной модели на тестовых данных.

In [19]:
# Оценка качества модели
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse},\
      \n MAE: {mae},\
      \n R2: {r2}')

MSE: 0.6629874283048177,      
 MAE: 0.6059780051879192,      
 R2: 0.4940606792889837
